# 1. Monolith DSCT

Demo of DSCT 

# 2. Library Import

In [1]:
%matplotlib widget

In [2]:
#General Libraries
import os
import time
import glob
import copy
import tomopy
import pybaselines
import numpy as np
import pandas as pd
import xarray as xr
import PIL
from PIL import Image
import fabio
from scipy.ndimage import median_filter
#Matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import subplots
#pyFAI
import pyFAI
from pyFAI.gui import jupyter
import fabio
from pyFAI.test.utilstest import UtilsTest
from pyFAI.calibrant import CALIBRANT_FACTORY
from pyFAI.goniometer import SingleGeometry
from pyFAI.gui.jupyter.calib import Calibration
print(f'Using pyFAI version: {pyFAI.version}')

UNSUPPORTED (log once): buildComputeProgram: cl2Metal failed


Using pyFAI version: 2024.5.0


'#diffpy\nimport diffpy\nimport diffpy.pdfgetx\nfrom diffpy.pdfgetx import PDFGetter, PDFConfig, loadData, Transformation'

In [3]:
class DSCT:
    #the init method
    def __init__(self, name):
        self.name=name
        
    #1st instance variable 
    def set_data(self, path):
        self.path = path
        
    def get_data(self):
        return([os.path.basename(x) for x in sorted(glob.glob(self.path+"*.nc"))])
        
    
    #2nd instance variable
    def config(self, calibrant, mask):
        self.calibrant = pyFAI.load(calibrant)
        self.mask = fabio.open(mask).data
        return(self.calibrant)

    #3rd instance variable
    def integrate(self, 
                  radial_range, #pass as 2-list e.g. [0,24]
                  delta_q, #stepsize e.g. 0.0025
                  data
                 ):
        
        self.radial_range=radial_range
        self.delta_q=delta_q
        q = []
        dsct = []
        phi = []
        x = []

        for i in [self.path + x for x in data]:
            with xr.open_dataset(i) as ds:
                ds.load()
            
            print(ds.attrs['save_name'])    
            phi_value = ds.attrs['mPhi']
            phi.append(phi_value)
            
            x_value = ds['mBaseX'].data
            x.append(x_value)
            
            intensities = []
            
            npt = int(np.ceil((self.radial_range[1] - self.radial_range[0]) / self.delta_q ))
            radial_range = [self.radial_range[0],self.radial_range[0]+self.delta_q*npt]
            
            for j in x_value:
                img = (ds.pe1_imgs.sel(mBaseX=j,method='nearest').astype('float32') - ds.pe1_img_dark.astype('float32')).values
                img = median_filter(img, size=2)
                
                i2d = self.calibrant.integrate2d(data=img, npt_rad=npt, 
                                                 npt_azim=360,correctSolidAngle=True, 
                                                 radial_range=self.radial_range,
                                                 mask=self.mask,dummy=np.NaN,
                                                 method='bbox',unit='q_A^-1',
                                                 safe=True, normalization_factor=1.0
                                                )
                da2d = xr.DataArray(data=i2d.intensity.astype('float32'),
                                    coords=[i2d.azimuthal.astype('float32'),i2d.radial.astype('float32')],
                                    dims=['azimuthal','radial'])
                da1d = da2d.mean(dim='azimuthal')
                q.append(da1d['radial'].data.tolist())
                intensities.append(da1d.data.tolist())

            intensities = [z for _, z in sorted(zip(x_value, intensities))]
            dsct.append(intensities)
        
        q_space = np.array(q[0], dtype='f8')
        i_obs = np.array(dsct, dtype='f8')
        phi_obs = np.array(phi, dtype='f8')
        x_obs = np.array(x[0], dtype='f8')
        
        DSCT = xr.Dataset(coords=dict(Q = ("Q", q_space), #q-space for xrd data in inverse angstroms
                                      Phi = ("Phi", phi_obs), #rotational angle 
                                      X = ("X", x_obs) #translational x position
                                     ),
                          data_vars=dict(XRD_CT = (["Phi", "X", "Q"], i_obs), #XRD-CT values
                                        ),
                          attrs=dict(Q_coordinate = "Inverse Angstroms",
                                     Phi_coordinate = "rotational degrees",
                                     X_coordinate = "translation position",
                                     XRDCT_variable = "Q-space CT values"
                                    ),
                         )
                
        return(DSCT)

# User Input

In [11]:
xrdct_path = '/Volumes/ONolan 2TB/2024-3-XPD-data/04-overnight-MOF/data/XRD-CT/'

# Data Importing, Integration, and Output to Tomography .nc File

Import each .nc file, azimuthally integrate each ['px','py','mbase-x'] array and store in a .nc file suitable for tomographic reconstruction

## 14503-37  - Crystalline Monolith XRD-CT

In [93]:
xmono_xrdct_project = DSCT('14503-37 XRD-CT')

In [95]:
xmono_xrdct_project.set_data(xrdct_path)
xmono_xrdct_project_file_list = xmono_xrdct_project.get_data()

In [97]:
xmono_xrdct_project_file_list

[]

In [ ]:
xmono_xrdct_project_file_list1 = xmono_xrdct_project_file_list[11:]
xmono_xrdct_project_file_list1

In [ ]:
xmono_xrdct_project_configuration = xmono_xrdct_project.config(xrd_ai_file,xrd_mask_file)
xmono_xrdct_project_configuration

In [ ]:
xmono_xrdct_project_integration = xmono_xrdct_project.integrate(radial_range=[0.5,9.5],delta_q=0.005, data=xmono_xrdct_project_file_list1) #2240 10-4-2024
xmono_xrdct_project_integration

In [ ]:
xmono_xrdct_project_integration.to_netcdf(out_path+'20231211_Dan_14503-37_CrystallineMonolith_XRDCT.nc')